In [1]:
%load_ext lab_black
%cd ../../..

/data3/SIG/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver8 import DatasetGeneratorVer8
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver8 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_3"
outdir = Path("results/distilbert-base-uncased-ver8")
fold = 3

In [5]:
with open("config/distilbert-base-uncased-ver8.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [6]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased-ver8'),
 'debug': False,
 'seed': 20210425,
 'ver': 8,
 'train': {'SAM': False,
  'folds': [1],
  'checkpoints': [None],
  'loss': {'name': 'focal',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 12,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ver6'),
  'batch_size': 30,
  'num_workers': 8,
  'oversampling': True,
  'oversampling_scale': 10}}

In [7]:
trainer = MyTrainer(C, fold, outdir / f"{postfix}.pth")

Load pretrained results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_3.pth


In [8]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [9]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [11]:
model.module.pre_classifier.register_forward_hook(hook)

In [12]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [13]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

100%|██████████████████████████████████████████████████████| 421079/421079 [46:25<00:00, 151.19it/s]


In [14]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [15]:
deck["tlevel"].shape, deck["fclevel"].shape

(torch.Size([421079]), torch.Size([421079]))

In [16]:
deck["feat"].shape

torch.Size([421079, 768])

In [17]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [ ]:
deck = torch.load(outdir / f"{postfix}-deck1.pth")

In [18]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [19]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[ -2.4621,   5.3335,  -9.9651,  -6.1716, -13.1533,  -5.5409, -12.0086]])


In [20]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[36.3845, 40.0326, 44.9746, 45.2894]]),
 tensor([[287, 437, 708, 579]]),
 tensor([[0, 3, 1, 0]]))

In [21]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-2.0527, -3.7650, -9.2502,  1.6917, -9.3212, -2.0611, -8.0563]])


In [22]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[46.1895, 49.0480, 50.7854, 54.4772]]),
 tensor([[268, 708, 372, 296]]),
 tensor([[0, 1, 1, 1]]))

In [23]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[  3.4239,  -0.4558, -11.1326,  -4.1132, -12.7674,  -4.5876, -11.5279]])


In [24]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[34.8809, 40.5718, 41.7815, 45.6098]]),
 tensor([[632, 579, 437, 396]]),
 tensor([[0, 0, 3, 1]]))

# 덱만들기 (Submission)

In [25]:
ds_test = trainer.dsgen.test()

In [26]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

100%|███████████████████████████████████████████████████| 1095951/1095951 [3:06:35<00:00, 97.89it/s]


In [27]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [28]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 1, 768]), torch.Size([1095951, 1]))

In [29]:
deck["feat"] = deck["feat"][:, 0, :]
# deck["fclevel"] = deck["fclevel"][:, 0]

In [30]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 768]), torch.Size([1095951, 1]))

In [31]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")